In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 23103169
paper_name = 'qian_zhang_2012' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/mmc2.xls', sheet_name='fitness_combined', skiprows=1)

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4449 x 22


In [7]:
original_data = original_data[['ORF','YPD rep1 fitness','YPD rep2 fitness','YPG fitness','YPE fitness','SC fitness','OAK fitness','ETH fitness']]

In [8]:
original_data.head()

,ORF,YPD rep1 fitness,YPD rep2 fitness,YPG fitness,YPE fitness,SC fitness,OAK fitness,ETH fitness
0,YAL001C,0.968989,0.990349,0.887079,0.960186,1.013147,0.867756,0.999240
1,YAL002W,0.918527,0.900021,0.923468,0.895325,0.948552,0.159949,0.963964
2,YAL004W,1.008346,1.009375,0.951429,0.960195,1.037183,0.811660,1.005849
3,YAL005C,0.996644,1.003788,0.937373,0.919715,1.030085,0.788691,1.001668
4,YAL007C,0.990941,1.004294,1.012650,1.018167,1.027506,0.947705,1.007461


In [9]:
original_data['ORF'] = original_data['ORF'].astype(str)

In [10]:
# Eliminate all white spaces & capitalize
original_data['ORF'] = clean_orf(original_data['ORF'])

In [11]:
# Translate to ORFs 
original_data['ORF'] = translate_sc(original_data['ORF'], to='orf')

In [12]:
# Make sure everything translated ok
t = looks_like_orf(original_data['ORF'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF, YPD rep1 fitness, YPD rep2 fitness, YPG fitness, YPE fitness, SC fitness, OAK fitness, ETH fitness]
Index: []


In [13]:
original_data['YPD fitness'] = original_data[['YPD rep1 fitness','YPD rep2 fitness']].mean(axis=1)

In [14]:
original_data.drop(columns=['YPD rep1 fitness','YPD rep2 fitness'], inplace=True)

In [15]:
original_data.set_index('ORF', inplace=True)
original_data.index.name = 'orf'

In [16]:
original_data = original_data.groupby(original_data.index).mean()

In [17]:
original_data.head()

,YPG fitness,YPE fitness,SC fitness,OAK fitness,ETH fitness,YPD fitness
orf,,,,,,
YAL001C,0.887079,0.960186,1.013147,0.867756,0.999240,0.979669
YAL002W,0.923468,0.895325,0.948552,0.159949,0.963964,0.909274
YAL004W,0.951429,0.960195,1.037183,0.811660,1.005849,1.008860
YAL005C,0.937373,0.919715,1.030085,0.788691,1.001668,1.000216
YAL007C,1.012650,1.018167,1.027506,0.947705,1.007461,0.997618


In [18]:
original_data.shape

(4424, 6)

# Prepare the final dataset

In [19]:
data = original_data.copy()

In [20]:
dataset_ids = [16489, 16488, 16490, 16492, 16491, 16487]
datasets = datasets.reindex(index=dataset_ids)

In [21]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [22]:
data.head()

dataset_id,16489,16488,16490,16492,16491,16487
data_type,value,value,value,value,value,value
orf,,,,,,
YAL001C,0.887079,0.960186,1.013147,0.867756,0.999240,0.979669
YAL002W,0.923468,0.895325,0.948552,0.159949,0.963964,0.909274
YAL004W,0.951429,0.960195,1.037183,0.811660,1.005849,1.008860
YAL005C,0.937373,0.919715,1.030085,0.788691,1.001668,1.000216
YAL007C,1.012650,1.018167,1.027506,0.947705,1.007461,0.997618


## Subset to the genes currently in SGD

In [23]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 21


In [24]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16489,16488,16490,16492,16491,16487
,data_type,value,value,value,value,value,value
gene_id,orf,,,,,,
1,YAL001C,0.887079,0.960186,1.013147,0.867756,0.999240,0.979669
2,YAL002W,0.923468,0.895325,0.948552,0.159949,0.963964,0.909274
1863,YAL004W,0.951429,0.960195,1.037183,0.811660,1.005849,1.008860
4,YAL005C,0.937373,0.919715,1.030085,0.788691,1.001668,1.000216
5,YAL007C,1.012650,1.018167,1.027506,0.947705,1.007461,0.997618


# Normalize

In [25]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [26]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [27]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,16489,16488,16490,16492,16491,16487,16489,16488,16490,16492,16491,16487
,data_type,value,value,value,value,value,value,valuez,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,,,
1,YAL001C,0.887079,0.960186,1.013147,0.867756,0.999240,0.979669,-0.771201,-0.293547,0.095695,-0.831484,-0.050607,-0.208412
2,YAL002W,0.923468,0.895325,0.948552,0.159949,0.963964,0.909274,-0.525830,-0.696229,-0.477261,-4.714370,-0.292987,-1.040578
1863,YAL004W,0.951429,0.960195,1.037183,0.811660,1.005849,1.008860,-0.337290,-0.293493,0.308887,-1.139216,-0.005194,0.136668
4,YAL005C,0.937373,0.919715,1.030085,0.788691,1.001668,1.000216,-0.432065,-0.544805,0.245934,-1.265223,-0.033925,0.034477
5,YAL007C,1.012650,1.018167,1.027506,0.947705,1.007461,0.997618,0.075530,0.066419,0.223056,-0.392900,0.005882,0.003764


# Print out

In [28]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [29]:
from IO.save_data_to_db3 import *

In [30]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/6 [00:00<?, ?it/s]

Deleting all datasets for PMID 23103169...
Inserting the new data...


100%|██████████| 6/6 [00:41<00:00,  6.95s/it]

Updating the data_modified_on field...
